In [ ]:
!pip install psycopg2
!pip install PyQt5

In [1]:
import sys
import psycopg2
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *

def dohvati_bazu():
    return psycopg2.connect(
        database="projekt",
        host="localhost",
        user="foi",
        password="vjezbe",
        port="5432"
    )

class Aplikacija(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Aplikacija")
        self.setFixedSize(QSize(800, 600))
        self.prikazi_izbornik()

    def prikazi_izbornik(self):
        layout = QGridLayout()

        gumb_usluge = QPushButton("1. Usluge", self)
        gumb_usluge.setStyleSheet("font-size: 45px;")
        gumb_usluge.clicked.connect(self.prikazi_usluge)
        gumb_usluge.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)

        gumb_cijene = QPushButton("2. Cijene", self)
        gumb_cijene.setStyleSheet("font-size: 45px;")
        gumb_cijene.clicked.connect(self.prikazi_cijene)
        gumb_cijene.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)

        gumb_trajanja = QPushButton("3. Trajanja", self)
        gumb_trajanja.setStyleSheet("font-size: 45px;")
        gumb_trajanja.clicked.connect(self.prikazi_trajanja)
        gumb_trajanja.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)

        gumb_troskovi = QPushButton("4. Troškovi", self)
        gumb_troskovi.setStyleSheet("font-size: 45px;")
        gumb_troskovi.clicked.connect(self.prikazi_troskove)
        gumb_troskovi.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)

        layout.addWidget(gumb_usluge, 0, 0)
        layout.addWidget(gumb_cijene, 0, 1)
        layout.addWidget(gumb_trajanja, 1, 0)
        layout.addWidget(gumb_troskovi, 1, 1)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

    def dohvati_usluge(self):
        try:
            conn = dohvati_bazu()
            cur = conn.cursor()
            cur.execute("SELECT uslugaID, naziv FROM usluga")
            usluge = cur.fetchall()
            cur.close()
            conn.close()
            return usluge
        except Exception as e:
            QMessageBox.warning(self, "Greška", f"Error: {e}")
            return []

    def dohvati_tablicu(self, upit, tablica):
        try:
            conn = dohvati_bazu()
            cur = conn.cursor()
            cur.execute(upit)
            redci = cur.fetchall()
            for redak in redci:
                ID = tablica.rowCount()
                tablica.insertRow(ID)
                for stupac, podatak in enumerate(redak):
                    tablica.setItem(ID, stupac, QTableWidgetItem(str(podatak)))
            cur.close()
            conn.close()
        except Exception as e:
            QMessageBox.warning(self, "Greška", f"Error: {e}")

    def dodaj_uslugu(self):
        naziv, opis, datum = self.inputs
        naziv = naziv.text()
        opis = opis.text()
        datum = datum.date().toString("yyyy-MM-dd")
        try:
            conn = dohvati_bazu()
            cur = conn.cursor()
            cur.execute("INSERT INTO usluga (naziv, opis, datum) VALUES (%s, %s, %s)",
                           (naziv, opis, datum))
            conn.commit()
            cur.close()
            conn.close()
            self.prikazi_usluge()
        except Exception as e:
            QMessageBox.warning(self, "Greška", f"Error: {e}")

    def dodaj_cijenu(self):
        usluge, rata, datum = self.inputs
        usluga = usluge.currentData()
        rata = rata.text()
        datum = datum.date().toString("yyyy-MM-dd")
        try:
            conn = dohvati_bazu()
            cur = conn.cursor()
            cur.execute("INSERT INTO cijena (uslugaID, rata, datum) VALUES (%s, %s, %s)",
                           (usluga, rata, datum))
            conn.commit()
            cur.close()
            conn.close()
            self.prikazi_cijene()
        except Exception as e:
            QMessageBox.warning(self, "Greška", f"Error: {e}")

    def dodaj_trajanje(self):
        usluge, pocetak, zavrsetak = self.inputs
        usluga = usluge.currentData()
        pocetak = pocetak.dateTime().toString("yyyy-MM-dd HH:mm:ss")
        zavrsetak = zavrsetak.dateTime().toString("yyyy-MM-dd HH:mm:ss")
        try:
            conn = dohvati_bazu()
            cur = conn.cursor()
            cur.execute("INSERT INTO trajanje (uslugaID, pocetak, zavrsetak) VALUES (%s, %s, %s)",
                           (usluga, pocetak, zavrsetak))
            conn.commit()
            cur.close()
            conn.close()
            self.prikazi_trajanja()
        except Exception as e:
            QMessageBox.warning(self, "Greška", f"Error: {e}")

    def stvori_gumb_dodaj(self, layout, atributi, funkcija):
        box_layout = QHBoxLayout()
        self.inputs = []

        for atribut in atributi:
            if atribut == "usluga":
                dropdown = QComboBox(self)
                usluge = self.dohvati_usluge()
                for ID, naziv in usluge:
                    dropdown.addItem(naziv, ID)
                box_layout.addWidget(dropdown)
                self.inputs.append(dropdown)
            elif atribut == "datum":
                datum = QDateEdit(self)
                datum.setCalendarPopup(True)
                datum.setDisplayFormat("yyyy-MM-dd")
                datum.setDate(QDate.currentDate())
                box_layout.addWidget(datum)
                self.inputs.append(datum)
            elif (atribut == "pocetak" or atribut == "zavrsetak"):
                vrijeme = QDateTimeEdit(self)
                vrijeme.setCalendarPopup(True)
                vrijeme.setDisplayFormat("yyyy-MM-dd HH:mm:ss")
                vrijeme.setDateTime(QDateTime.currentDateTime())
                box_layout.addWidget(vrijeme)
                self.inputs.append(vrijeme)
            else:
                tekst = QLineEdit(self)
                tekst.setPlaceholderText(atribut)
                box_layout.addWidget(tekst)
                self.inputs.append(tekst)

        gumb = QPushButton("Dodaj", self)
        gumb.clicked.connect(funkcija)
        box_layout.addWidget(gumb)
        layout.addLayout(box_layout)

    def izbrisi_redak(self, tablica_naziv, tablica_ID, prozor):
        tablica = self.findChild(QTableWidget)
        redak = tablica.currentRow()
        if redak == -1:
            QMessageBox.warning(self, "Greška", "Odaberite redak za brisanje.")
            return
        ID = tablica.item(redak, 3).text()
        
        try:
            conn = dohvati_bazu()
            cur = conn.cursor()
            cur.execute(f"DELETE FROM {tablica_naziv} WHERE {tablica_ID} = %s", (ID,))
            conn.commit()
            cur.close()
            conn.close()
            prozor()
        except Exception as e:
            QMessageBox.warning(self, "Greška", f"Error: {e}")

    def stvori_gumb_izbrisi(self, layout, tablica, ID, prozor):
        gumb = QPushButton("Izbriši odabrani redak", self)
        gumb.clicked.connect(lambda: self.izbrisi_redak(tablica, ID, prozor))
        layout.addWidget(gumb)

    def stvori_gumb_povratak(self, layout):
        gumb = QPushButton("<--- Povratak", self)
        gumb.clicked.connect(self.prikazi_izbornik)
        layout.addWidget(gumb)

    def prikazi_usluge(self):
        layout = QVBoxLayout()

        self.tablica = QTableWidget(self)
        self.tablica.setColumnCount(4)
        self.tablica.setColumnWidth(1, 300)
        self.tablica.setColumnHidden(3, True)
        self.tablica.setHorizontalHeaderLabels(["Naziv", "Opis", "Datum", "uslugaID"])
        self.dohvati_tablicu("SELECT naziv, opis, datum, uslugaID FROM usluga", self.tablica)

        self.stvori_gumb_dodaj(layout, ["Unesi naziv", "Unesi opis", "datum"], self.dodaj_uslugu)
        layout.addWidget(self.tablica)
        self.stvori_gumb_izbrisi(layout, "usluga", "uslugaID", self.prikazi_usluge)
        self.stvori_gumb_povratak(layout)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

    def prikazi_cijene(self):
        layout = QVBoxLayout()

        self.tablica = QTableWidget(self)
        self.tablica.setColumnCount(4)
        self.tablica.setColumnHidden(3, True)
        self.tablica.setHorizontalHeaderLabels(["Usluga", "Rata (€/h)", "Datum", "cijenaID"])
        self.dohvati_tablicu("""
            SELECT u.naziv, c.rata, c.datum, c.cijenaID
            FROM cijena c
            JOIN usluga u ON c.uslugaID = u.uslugaID
        """, self.tablica)

        self.stvori_gumb_dodaj(layout, ["usluga", "Unesi ratu po satu", "datum"], self.dodaj_cijenu)
        layout.addWidget(self.tablica)
        self.stvori_gumb_izbrisi(layout, "cijena", "cijenaID", self.prikazi_cijene)
        self.stvori_gumb_povratak(layout)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

    def prikazi_trajanja(self):
        layout = QVBoxLayout()

        self.tablica = QTableWidget(self)
        self.tablica.setColumnCount(4)
        self.tablica.setColumnWidth(1, 150)
        self.tablica.setColumnWidth(2, 150)
        self.tablica.setColumnHidden(3, True)
        self.tablica.setHorizontalHeaderLabels(["Usluga", "Početak", "Završetak", "trajanjeID"])
        self.dohvati_tablicu("""
            SELECT u.naziv, t.pocetak, t.zavrsetak, t.trajanjeID
            FROM trajanje t
            JOIN usluga u ON t.uslugaID = u.uslugaID
        """, self.tablica)

        self.stvori_gumb_dodaj(layout, ["usluga", "pocetak", "zavrsetak"], self.dodaj_trajanje)
        layout.addWidget(self.tablica)
        self.stvori_gumb_izbrisi(layout, "trajanje", "trajanjeID", self.prikazi_trajanja)
        self.stvori_gumb_povratak(layout)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

    def prikazi_troskove(self):
        layout = QVBoxLayout()

        self.tablica = QTableWidget(self)
        self.tablica.setColumnCount(3)
        self.tablica.setColumnWidth(1, 150)
        self.tablica.setHorizontalHeaderLabels(["Usluga", "Trajanje", "Ukupno (€)"])
        self.dohvati_tablicu("""
            SELECT u.naziv, (t.zavrsetak - t.pocetak)
            AS vrijeme, v.ukupno
            FROM trosak v
            JOIN usluga u ON v.uslugaID = u.uslugaID
            JOIN trajanje t ON v.trajanjeID = t.trajanjeID;
        """, self.tablica)

        layout.addWidget(self.tablica)
        self.stvori_gumb_povratak(layout)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

app = QApplication(sys.argv)
window = Aplikacija()
window.show()
app.exec()

MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen


0